# Doc2Vec Model

In [1]:
import re
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from scipy import spatial
from pmaw import PushshiftAPI
from pathlib import Path

In [26]:
reddit_data_path = Path("data/reddit_data.txt")
essais_politiques_path = Path("data/quinze_essais_politiques.txt")
model_path = "models/doc2vec_model"

In [4]:
#scrapping reddit data and store it in a txt file

def get_reddit_data(nbPost):
    api = PushshiftAPI()
    comments = api.search_submissions(subreddit="Politiquefrancaise", limit=nbPost, TimeoutError=1000)
    
    comment_list = [comment['selftext'] for comment in comments]
    with open(reddit_data_path, "w", encoding="utf-8") as f:
        for comment in comment_list:
            #remove urls
            comment = re.sub(r'http\S+', '', comment)
            f.write(comment)
    return

if not reddit_data_path.exists():
    print("Creating reddit data file")
    get_reddit_data(10000)
else:
    print("Data already exists")


Data already exists


In [19]:
#télcharge "quinze essais politiques" dans data et en fait une liste
# opening the file in read mode
#my_file = open("data/quinze_essais_politiques.txt", "r") -> not working


with open(reddit_data_path, "r", encoding="utf-8") as f:
    corpus = f.read()
    
    corpus = corpus.replace('\n', ' ')
    corpus = corpus.replace('.', ',')

    # replacing end splitting the text 
    # when newline ('\n') is seen.
    corpus = corpus.split(",")
    
print(len(corpus))
for sentence in corpus[:10]:
    print(sentence)
print(corpus[:10])

4085
Le président Emmanuel Macron a appelé dans un discours devant le Parlement européen à proposer une nouvelle alliance avec les pays africains en matière d'investissements
 de santé et de sécurité
  Macron a annoncé que la France organisera un sommet Europe-Afrique en février
 dans le but de reconstruire le partenariat entre les deux continents
  Cet appel est venu dans un discours dans lequel il a passé en revue les priorités de la France à Strasbourg à l'occasion de sa présidence de l'Union européenne au cours du premier semestre 2022
 dans lequel il a évoqué la démocratie
 l'environnement et la paix dans le vieux continent
  Macron a souligné que les Européens devaient établir un "nouvel ordre de sécurité et de stabilité" qui appelle à un "réarmement stratégique" et à des "pourparlers francs" avec la Russie
  [  Le Premier ministre Jean Castex a annoncé il y a quelques jours
 lors d'une conférence de presse conjointe avec le ministre de la Santé Olivier Veran
["Le président Emman

Use re module to preprocess data

Convert all letters into lowercase

Remove punctuations, numbers, etc

In [20]:
for i in range(len(corpus)):
    corpus[i] = corpus[i].lower()
    #remove punctuation
    corpus[i] = re.sub(r'[^\w\s]','',corpus[i])
    #make a list of corpus[i]
    corpus[i] = corpus[i].split()
    

corpus[2]


['macron',
 'a',
 'annoncé',
 'que',
 'la',
 'france',
 'organisera',
 'un',
 'sommet',
 'europeafrique',
 'en',
 'février']

For the doc2vec model, input data should be in format of iterable TaggedDocuments"

Each TaggedDocument instance comprises words and tags

Hence, each document (i.e., a sentence or paragraph) should have a unique tag which is identifiable

In [21]:
for i in range(len(corpus)):
    corpus[i] = TaggedDocument(words = corpus[i], tags = [f'sent{i}'])    # converting each sentence into a TaggedDocument
corpus[2]

TaggedDocument(words=['macron', 'a', 'annoncé', 'que', 'la', 'france', 'organisera', 'un', 'sommet', 'europeafrique', 'en', 'février'], tags=['sent2'])

In [27]:
model = Doc2Vec(documents = corpus, vector_size = 10, min_count = 1)
model.init_sims(replace = True)

model.save(model_path)
model = Doc2Vec.load(model_path)


C:\Users\antoi\AppData\Local\Temp/ipykernel_22320/122182469.py:2: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace = True)


In [54]:
v1 = model.infer_vector([' front national'])    # in doc2vec, infer_vector() function is used to infer the vector embedding of a document
v2 = model.infer_vector(['insoumis'])    # in doc2vec, infer_vector() function is used to infer the vector embedding of a document
# define a function that computes cosine similarity between two words
def cosine_similarity(v1, v2):
    return 1 - spatial.distance.cosine(v1, v2)
cosine_similarity(v1, v2)






-0.09528177231550217